In [6]:
#set up environment
import numpy as np
import pandas as pd 
import re
from bs4 import BeautifulSoup
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed
from keras.models import Model
from keras.callbacks import EarlyStopping
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")

In [ ]:
!git clone https://github.com/complementizer/wcep-mds-dataset

In [ ]:
cd wcep-mds-dataset/experiments

In [ ]:
!mkdir WCEP
!gdown https://drive.google.com/uc?id=1kUjSRXzKnTYdJ732BkKVLg3CFxDKo25u -O WCEP/train.jsonl.gz
!gdown https://drive.google.com/uc?id=1_kHTZ32jazTbXaFRg0vBeIsVcpI7CTmy -O WCEP/val.jsonl.gz
!gdown https://drive.google.com/uc?id=1qsd5pOCpeSXsaqNobXCrcAzhcjtG1wA1 -O WCEP/test.jsonl.gz

In [2]:
#load dataset
import json, gzip
def read_jsonl_gz(path):
    with gzip.open(path) as f:
        for l in f:
            yield json.loads(l)

dataset = list(read_jsonl_gz('./WCEP/train.jsonl.gz')) + list(read_jsonl_gz('./WCEP/test.jsonl.gz'))

In [3]:
def get_shortest_article(event):
    """
    Input: event from dataset
    Output: shortest article describing the event (performance is better on shorter articles)
    """
    num_articles = len(event['articles'])
    article_list = [event['articles'][i]['text'] for i in range(num_articles) if len(event['articles'][i]['text']) > 0]
    return min(article_list, key=len) 

In [4]:
#wrap data in a data frame
articles  = list()
summaries = list()
for event in dataset:
    shortest_article = get_shortest_article(event)
    articles.append(shortest_article)
    summaries.append(event['summary'])
data = pd.DataFrame(data={'articles': articles, 'summaries': summaries})

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9180 entries, 0 to 9179
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   articles   9180 non-null   object
 1   summaries  9180 non-null   object
dtypes: object(2)
memory usage: 143.6+ KB
